In [1]:
# Importing dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

In [2]:
executable_path = {'executable_path':ChromeDriverManager().install()}
browser = Browser('chrome',**executable_path, headless=False)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\tamer\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache


In [3]:
# Defining a dictionary to collect all scraped data
mars_dict = {} 

# Scraping 'Nasa Mars News' site
nasa_url = 'https://redplanetscience.com/'
browser.visit(nasa_url)



html = browser.html
soup = bs(html, 'html.parser')

news_title = soup.find('div', class_= 'content_title').text
news_p = soup.find('div', class_= 'article_teaser_body').text

# Adding 'Nasa Mars News' data to mars_dict
mars_dict['news_title'] = news_title
mars_dict['news_p'] = news_p


In [4]:
# Scraping 'JPL Mars Space Images' featured image
jpl_url = 'https://spaceimages-mars.com/'
browser.visit(jpl_url)

html = browser.html
soup = bs(html, 'html.parser')

featured_image = soup.find('img', class_= 'headerimage')
relative_image_url = featured_image['src']
featured_image_url = jpl_url+relative_image_url

mars_dict['featured_image_url'] = featured_image_url

In [5]:
# Scraping 'Mars Facts' using pandas
facts_url = 'https://galaxyfacts-mars.com/'

tables_list = pd.read_html(facts_url)

# Required table is the first table in the list
table_df = tables_list[0]

table_df.columns = ['Description', 'Mars', 'Earth']
table_df.set_index('Description', inplace=True)


# Save the table to an html file
html_table = table_df.to_html()
html_table = html_table.replace('\n','')
html_table = html_table.replace('class="dataframe', 'class="table table-striped table-bordered')
html_table = html_table.replace('style="text-align: right;"', 'style="text-align: center;"')

# Adding 'Mars Facts' table to mars_dict
mars_dict['facts'] = html_table
html_table

'<table border="1" class="table table-striped table-bordered">  <thead>    <tr style="text-align: center;">      <th></th>      <th>Mars</th>      <th>Earth</th>    </tr>    <tr>      <th>Description</th>      <th></th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Mars - Earth Comparison</th>      <td>Mars</td>      <td>Earth</td>    </tr>    <tr>      <th>Diameter:</th>      <td>6,779 km</td>      <td>12,742 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg</td>      <td>5.97 × 10^24 kg</td>    </tr>    <tr>      <th>Moons:</th>      <td>2</td>      <td>1</td>    </tr>    <tr>      <th>Distance from Sun:</th>      <td>227,943,824 km</td>      <td>149,598,262 km</td>    </tr>    <tr>      <th>Length of Year:</th>      <td>687 Earth days</td>      <td>365.24 days</td>    </tr>    <tr>      <th>Temperature:</th>      <td>-87 to -5 °C</td>      <td>-88 to 58°C</td>    </tr>  </tbody></table>'

In [6]:
# Scraping 'Mars Hemispheres'
hem_url = 'https://marshemispheres.com/'
browser.visit(hem_url)

html = browser.html
soup = bs(html, 'html.parser')

# Getting a list for the links to be clicked
clickables = soup.find_all('a', class_='itemLink product-item')

# clickables contain duplicate links, keep only the ones that contain text
links = []
for link in clickables:
    if link.text:
        links.append(link)
        
# Delete unneeded last element of links
links.pop()

# Defining a list to collect data
hemisphere_image_urls = []

for link in links:
    hem_dict = {}
    link_text = link.find('h3').text
    hem_dict['title'] = link_text
    #Click to visit the image page
    browser.links.find_by_partial_text(link_text).click()
    time.sleep(1)
    html = browser.html
    soup = bs(html, 'html.parser')
    img_section = soup.find('div', class_='downloads')
    hem_dict['img_url'] = hem_url + img_section.find('a', text='Original')['href']
    hemisphere_image_urls.append(hem_dict)
    # Going back to the previous page
    browser.back()
    time.sleep(1)

browser.quit()

mars_dict['hemishperes'] = hemisphere_image_urls